In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17265, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 17265 (delta 57), reused 14 (delta 14), pack-reused 17160 (from 4)
Receiving objects: 100% (17265/17265), 16.01 MiB | 11.64 MiB/s, done.
Resolving deltas: 100% (11799/11799), done.


In [2]:
%cd yolov5

/Users/admlanno/Desktop/Курсовая/yolov5


In [3]:
!pip install -U -r requirements.txt

  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 15.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.2/910.2 kB 14.1 MB/s eta 0:00:00
Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.5
    Uninstalling tqdm-4.66.5:
      Successfully uninstalled tqdm-4.66.5
  Attempting uninstall: sympy
    Found exist

In [5]:
!python train.py --img 640 --batch 16 --epochs 50 --data /Users/admlanno/Desktop/Курсовая/dataset/dataset_test/data.yaml --cfg yolov5s.yaml --weights yolov5s.pt --name yolov5_model

train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=/Users/admlanno/Desktop/Курсовая/dataset/dataset_test/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-397-gde62f93c Python-3.12.2 torch-2.5.1 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_l

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Подгрузка данных
res = pd.read_csv('restored.csv')

activity_mapping = {
    "WALKING": 1,
    "WALKING_UPSTAIRS": 2,
    "WALKING_DOWNSTAIRS": 3,
    "SITTING": 4,
    "STANDING": 5,
    "LAYING": 6
}

res = res['matched_Activity'].map(activity_mapping).astype(int)
best_accuracy = 0
best_seed = 0
# Цикл по количеству нейронов
for sed in range(1, 100):

    class Model(nn.Module):
        def __init__(self, input_size):
            super(Model, self).__init__()

            self.fc1 = nn.Linear(input_size, 37)
            self.bn1 = nn.BatchNorm1d(37)
            self.relu1 = nn.ReLU()
            self.dropout1 = nn.Dropout(p=0)

            self.fc2 = nn.Linear(37, 7)
            self.bn2 = nn.BatchNorm1d(7)
            self.relu2 = nn.ReLU()
            self.dropout2 = nn.Dropout(p=0)

            self.output_layer = nn.Linear(7, 6)

        def forward(self, x):
            x = self.fc1(x)
            x = self.bn1(x)
            x = self.relu1(x)
            x = self.dropout1(x)

            x = self.fc2(x)
            x = self.bn2(x)
            x = self.relu2(x)
            x = self.dropout2(x)

            x = self.output_layer(x)
            return x

    set_seed(sed)
    torch.cuda.manual_seed(sed)

    # Convert numpy arrays to torch tensors
    tX_tensor = torch.tensor(tX, dtype=torch.float32)
    tY_tensor = torch.tensor(tY.values, dtype=torch.long)

    # If using GPU
    tX_tensor = tX_tensor.to(device)
    tY_tensor = tY_tensor.to(device)

    dataset = TensorDataset(tX_tensor, tY_tensor)
    val_size = int(len(dataset) * 0.3)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=512)

    model = Model(input_size=tX.shape[1]).to(device)
    print(model)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_losses = []
    val_losses = []
    accuracies = []

    # Training loop with data collection
    epochs = 30
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)


        # Validation loop
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            avg_val_loss = val_loss / len(val_loader)
            accuracy = 100 * correct / total
            val_losses.append(avg_val_loss)
            accuracies.append(accuracy)

    # Assuming vX is a numpy array containing your validation data
    # You need to convert it into a torch tensor with the correct type
    vX_tensor = torch.tensor(vX, dtype=torch.float32)

    # If using GPU, send the model and the tensor to GPU
    model.to('cuda')
    vX_tensor = vX_tensor.to('cuda')

    # Set the model to evaluation mode
    model.eval()

    # No need to compute gradients (for memory efficiency)
    with torch.no_grad():
        # Make predictions
        predictions = model(vX_tensor)

    # If you need class probabilities, apply softmax
    probabilities = torch.softmax(predictions, dim=1)

    # To get the predicted class labels, get the index of the max log-probability
    predicted_labels = torch.max(probabilities, 1)[1]

    # Convert to numpy array if needed (for further processing in non-PyTorch code)
    probabilities_np = probabilities.cpu().numpy()
    predicted_labels_np = predicted_labels.cpu().numpy()

    # Now 'probabilities_np' holds class probabilities and 'predicted_labels_np' holds class predictions
    YLab = [f'{i}/{s}' for i, s in enumerate('walking walking_upstairs walking_downstairs sitting standing laying'.split())]  # column labels
    pd.DataFrame(probabilities_np[:3,:], columns=YLab).style.background_gradient(cmap='coolwarm', axis=1)  # display first few predictions
    result = pd.DataFrame(predicted_labels_np + 1, columns=['y']) # labels are shifted to the initial state


    accuracy = accuracy_score(res, result["y"])
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        best_seed = sed
        print(f'best accuracy {best_accuracy} is for {sed}')
    print(f"Accuracy модели для {sed}: {accuracy:.4f}, best seed is {best_seed} with accuracy = {best_accuracy}")


SyntaxError: invalid syntax (3675908640.py, line 1)